In [1]:
import pandas as pd
import ta 
import optuna 
import time
import numpy as np
from multiprocessing import Pool
from itertools import combinations, chain 
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [2]:
data_1d_test = pd.read_csv("aapl_1d_test.csv")
data_1d_test = data_1d_test.dropna()

In [3]:
def powerset(s):
    return chain.from_iterable(combinations(s,r) for r in range(1,len(s)+1))

In [4]:
def file_features(data, ds_type: str):
    data1=pd.DataFrame()
    #Calcular indicadores tecnicos
    cmf_data = ta.volume.ChaikinMoneyFlowIndicator(data.High, data.Low, data.Close, data.Volume, window = 14)
    rsi_data = ta.momentum.RSIIndicator(data.Close, window=14)
    
    data1["CMF"] = cmf_data.chaikin_money_flow()
    data1["RSI"] = rsi_data.rsi()
    # Calcular la volatilidad
    data1['Volatility'] = data['High'] - data['Low']
    data1['Close_Lag0'] = data['Close']
    # Calcular las tendencias
    for i in range(1, 5 + 1):
        data1[f'Close_Lag{i}'] = data['Close'].shift(i)
    #Variable ded respuesta
    if ds_type == "buy":
        data1['Response'] = (data['Close'] < data['Close'].shift(-10))
    else:
        data1['Response'] = (data['Close'] > data['Close'].shift(-10))
    
    data1 = data1.drop(data1.index[:30])
    data1 = data1.drop(data1.index[-30:])
    data1.reset_index(drop=True, inplace=True)
    
    return data1

In [5]:
dataresult_long_1d_test = file_features(data_1d_test, ds_type="buy")
dataresult_short_1d_test = file_features(data_1d_test, ds_type="sell")

Como ya tenemos los parametros optimos, solo los aplicamos...

In [6]:
def buy_signals(data):
    buy_signals = pd.DataFrame()
    # Selecciona las características
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]

    # Crear modelos con los mejores parámetros encontrados para cada algoritmo
    best_logistic_model = LogisticRegression(penalty= 'l1', C= 0.008798139830459064, solver= 'saga')
    best_svm_model = SVC(C= 37.29423057590281, kernel= 'sigmoid', gamma= 'auto')
    best_xgboost_model = XGBClassifier(n_estimators= 100, max_depth= 8, learning_rate= 0.02353186669766063, subsample= 0.8, colsample_bytree= 1.0)

    # Entrenar los modelos con todo el conjunto de datos original
    best_logistic_model.fit(X, y)
    best_svm_model.fit(X, y)
    best_xgboost_model.fit(X, y)

    # Realizar predicciones en el conjunto de datos original
    predictions_lr = best_logistic_model.predict(X)
    predictions_svm = best_svm_model.predict(X)
    predictions_xgboost = best_xgboost_model.predict(X)
    predictions_xgboost_bool = predictions_xgboost.astype(bool)


    # Agregar las predicciones como nuevas columnas al conjunto de datos original
    buy_signals['predicciones_lr'] = predictions_lr
    buy_signals['predicciones_svm'] = predictions_svm
    buy_signals['predicciones_xgboost'] = predictions_xgboost_bool

    return buy_signals

In [7]:
def sell_signals(data):
    sell_signals = pd.DataFrame()
    # Selecciona las características
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]

    # Crear modelos con los mejores parámetros encontrados para cada algoritmo
    best_logistic_model = LogisticRegression(penalty='l1', C=391.6055943510264, solver='liblinear')
    best_svm_model = SVC(C=34.154290341054214, kernel='rbf', gamma='scale')
    best_xgboost_model = XGBClassifier(n_estimators=300, max_depth=7, learning_rate=0.01845611375279615, subsample=0.7,
                                       colsample_bytree=0.9)

    # Entrenar los modelos con todo el conjunto de datos original
    best_logistic_model.fit(X, y)
    best_svm_model.fit(X, y)
    best_xgboost_model.fit(X, y)

    # Realizar predicciones en el conjunto de datos original
    predictions_lr = best_logistic_model.predict(X)
    predictions_svm = best_svm_model.predict(X)
    predictions_xgboost = best_xgboost_model.predict(X)
    predictions_xgboost_bool = predictions_xgboost.astype(bool)

    # Agregar las predicciones como nuevas columnas al conjunto de datos original
    sell_signals['predicciones_lr'] = predictions_lr
    sell_signals['predicciones_svm'] = predictions_svm
    sell_signals['predicciones_xgboost'] = predictions_xgboost_bool

    return sell_signals

In [8]:
global_buy_signals = buy_signals(dataresult_long_1d_test)
global_sell_signals = sell_signals(dataresult_short_1d_test)

In [9]:
def backtest(data, buy_signals, sell_signals, stop_loss, take_profit, n_shares):
            history = []
            active_operations = []
            cash = 1_000_000
            com = 1.25 / 100

            for i, row in data.iterrows():
                # close active operation
                active_op_temp = []
                for operation in active_operations:
                    if operation["stop_loss"] > row.Close:
                        cash += (row.Close * operation["n_shares"]) * (1 - com)
                    elif operation["take_profit"] < row.Close:
                        cash += (row.Close * operation["n_shares"]) * (1 - com)
                    else:
                        active_op_temp.append(operation)
                active_operations = active_op_temp

                # check if we have enough cash
                if cash < (row.Close * (1 + com)):
                    asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
                    portfolio_value = cash + asset_vals
                    continue

                # Apply buy signals
                if buy_signals.loc[i].any():
                    active_operations.append({
                        "bought": row.Close,
                        "n_shares": n_shares,
                        "stop_loss": row.Close * stop_loss,
                        "take_profit": row.Close * take_profit
                    })

                    cash -= row.Close * (1 + com) * n_shares

                # Apply sell signals
                if sell_signals.loc[i].any():
                    active_op_temp = []
                    for operation in active_operations:
                        if operation["take_profit"] < row.Close or operation["stop_loss"] > row.Close:
                            cash += (row.Close * operation["n_shares"]) * (1 - com)
                        else:
                            active_op_temp.append(operation)
                    active_operations = active_op_temp

                asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
                portfolio_value = cash + asset_vals

            return portfolio_value

In [10]:
def optimize(trial, strategy, data):
    portfolio_value = 0

    stop_loss = trial.suggest_float("stop_loss", 0.80, 0.90)
    take_profit = trial.suggest_float("take_profit", 1.01, 1.10)
    n_shares = trial.suggest_int("n_shares", 20, 50)

    strat_params = {}

    buy_signals = pd.DataFrame()
    sell_signals = pd.DataFrame()

    if "logistic" in strategy:
        buy_signals["logistic"] = global_buy_signals["predicciones_lr"]
        sell_signals["logistic"] = global_sell_signals["predicciones_lr"]
        
    if "svm" in strategy:
        buy_signals["svm"] = global_buy_signals["predicciones_svm"]
        sell_signals["svm"] = global_sell_signals["predicciones_svm"]
        
    if "xg" in strategy:
        buy_signals["xg"] = global_buy_signals["predicciones_xgboost"]
        sell_signals["xg"] = global_sell_signals["predicciones_xgboost"]
    
    return backtest(data, buy_signals, sell_signals, stop_loss, take_profit, n_shares)

In [11]:
def optimize_file(data):
    data = data.drop(data.index[:30])
    data = data.drop(data.index[-30:])
    data.reset_index(drop=True, inplace=True)
    strategies = list(powerset(["logistic", "svm", "xg"]))
    best_strat = None
    best_val = -1
    best_params = None

    for strat in strategies:
        study = optuna.create_study(direction="maximize")
        study.optimize(lambda x: optimize(x, strat, data), n_trials=15)
        value = study.best_value
        if value > best_val:
            best_val = value
            best_strat = strat
            best_params = study.best_params
    print(study.best_value)
    print(best_strat)
    print(best_params)

    return {"file": data,
            "strat": best_strat,
            "value": best_val,
            "params": best_params}

In [12]:
file_1d_test = optimize_file(data_1d_test)

[I 2024-03-09 17:10:26,597] A new study created in memory with name: no-name-0d66db1f-627f-401e-9d97-e5a92781a25e
[I 2024-03-09 17:10:27,062] Trial 0 finished with value: 929448.4000000014 and parameters: {'stop_loss': 0.8615603013048945, 'take_profit': 1.0100357840348415, 'n_shares': 35}. Best is trial 0 with value: 929448.4000000014.
[I 2024-03-09 17:10:27,640] Trial 1 finished with value: 938208.306500002 and parameters: {'stop_loss': 0.8882869404542738, 'take_profit': 1.0535415550993186, 'n_shares': 34}. Best is trial 1 with value: 938208.306500002.
[I 2024-03-09 17:10:28,104] Trial 2 finished with value: 926560.0290000004 and parameters: {'stop_loss': 0.8633192481852381, 'take_profit': 1.0318740497943815, 'n_shares': 44}. Best is trial 1 with value: 938208.306500002.
[I 2024-03-09 17:10:28,875] Trial 3 finished with value: 957285.2243749994 and parameters: {'stop_loss': 0.8216103127085199, 'take_profit': 1.0516424109779754, 'n_shares': 43}. Best is trial 3 with value: 957285.22437

[I 2024-03-09 17:10:53,256] Trial 5 finished with value: 1048978.9072499978 and parameters: {'stop_loss': 0.8776743577921902, 'take_profit': 1.0621858081235662, 'n_shares': 37}. Best is trial 0 with value: 1069801.4720000022.
[I 2024-03-09 17:10:53,662] Trial 6 finished with value: 1047751.4620000005 and parameters: {'stop_loss': 0.8835276978631799, 'take_profit': 1.0677395315562128, 'n_shares': 33}. Best is trial 0 with value: 1069801.4720000022.
[I 2024-03-09 17:10:54,192] Trial 7 finished with value: 1080817.5164999985 and parameters: {'stop_loss': 0.8931521512527769, 'take_profit': 1.0939063742452575, 'n_shares': 38}. Best is trial 7 with value: 1080817.5164999985.
[I 2024-03-09 17:10:54,379] Trial 8 finished with value: 993127.9367500006 and parameters: {'stop_loss': 0.8783455621077678, 'take_profit': 1.0179895229227758, 'n_shares': 39}. Best is trial 7 with value: 1080817.5164999985.
[I 2024-03-09 17:10:54,729] Trial 9 finished with value: 1045565.4718750009 and parameters: {'sto

[I 2024-03-09 17:11:21,306] Trial 11 finished with value: 1026895.8475000015 and parameters: {'stop_loss': 0.8015333835709499, 'take_profit': 1.0951744355950546, 'n_shares': 20}. Best is trial 11 with value: 1026895.8475000015.
[I 2024-03-09 17:11:22,529] Trial 12 finished with value: 1028170.5100000018 and parameters: {'stop_loss': 0.801286328596874, 'take_profit': 1.0965268512395352, 'n_shares': 20}. Best is trial 12 with value: 1028170.5100000018.
[I 2024-03-09 17:11:23,680] Trial 13 finished with value: 1026411.5700000027 and parameters: {'stop_loss': 0.8177813661414028, 'take_profit': 1.0996447608103064, 'n_shares': 20}. Best is trial 12 with value: 1028170.5100000018.
[I 2024-03-09 17:11:25,071] Trial 14 finished with value: 1035567.8906249984 and parameters: {'stop_loss': 0.8003040865232879, 'take_profit': 1.099315463740782, 'n_shares': 25}. Best is trial 14 with value: 1035567.8906249984.
[I 2024-03-09 17:11:25,077] A new study created in memory with name: no-name-f3051f60-8c91

1050416.7555000053
('xg',)
{'stop_loss': 0.899950212993615, 'take_profit': 1.0987292727726978, 'n_shares': 47}
